In [1]:
import numpy as np
import scipy as sp

In [2]:
def rodrigez(fi,p): 
    e=np.eye(3)
    px=np.array([[0,-p[2],p[1]],
                 [p[2],0,-p[0]],
                 [-p[1],p[0],0]])
    ppT=np.array([[p[0]**2,p[0]*p[1],p[0]*p[2]]
                 ,[p[1]*p[0],p[1]**2,p[1]*p[2]]
                 ,[p[2]*p[0],p[2]*p[1],p[2]**2]])
    return (1-np.cos(fi))*ppT+np.cos(fi)*e+np.sin(fi)*px

In [3]:
def provera(A): # Provera za rotacije, tj. matrice kretanja
    provera=A@A.T
    provera=np.where(np.isclose(provera,0),0,provera)
    provera=np.where(np.isclose(provera,1),1,provera)
    if (A.shape[0]!=A.shape[1]) or (not np.isclose(np.linalg.det(A),1)) or not np.all(provera==np.identity(A.shape[0])):
        return "Nije matrica kretanja!"
    else:
        return "Sve ok!"

--------------------Odavde pocinje cetvrti domaci--------------------

In [6]:
import numpy as np
import scipy as sp
import math

class Quaternion:
    def __init__(self,params,w):
        self.i=params[0]
        self.j=params[1]
        self.k=params[2]
        self.params=params
        self.w=w
        
    def __truediv__(self,o:Quaternion)->Quaternion:
        if isinstance(o,Quaternion):
            donji=o.i**2+o.j**2+o.k**2+o.w**2
            return Quaternion([(o.w*self.i-o.i*self.w-o.j*self.k+o.k*self.j)/donji,
                              (o.w*self.j-o.i*self.k-o.j*self.w+o.k*self.i)/donji,
                              (o.w*self.k-o.i*self.j-o.j*self.i+o.k*self.w)/donji],
                              (o.w*self.w-o.i*self.i-o.j*self.j+o.k*self.k)/donji)
        else:
            a=[]
            for x in self.params:
                a.append(x/o)
            return Quaternion(a,self.w/o)
            
    def __add__(self,o:Quaternion)->Quaternion:
        if isinstance(o,Quaternion):
            return Quaternion([self.i+o.i,self.j+o.j,self.k+o.k],self.w+o.w)
        else:
            return Quaternion(self.params,self.w+o)
            
    def __radd__(self,o):
        if not isinstance(o,Quaternion) and not isinstance(o,str):
            return self.__add__(o)

    def __sub__(self,o:Quaternion)->Quaternion:
        if isinstance(o,Quaternion):
            return Quaternion([self.i-o.i,self.j-o.j,self.k-o.k],self.w-o.w)       
        else:
            return Quaternion(self.params,self.w-o)
            
    def __mul__(self,o:Quaternion)->Quaternion:
        if isinstance(o,Quaternion): 
            return Quaternion([self.w*o.i+self.i*o.w+self.j*o.k-self.k*o.j,
                               self.w*o.j+self.j*o.w+self.k*o.i-self.i*o.k,
                               self.w*o.k+self.k*o.w+self.i*o.j-self.j*o.i],
                               self.w*o.w-self.i*o.i-self.j*o.j-self.k*o.k)
        else:
            a=[]
            for x in self.params:
                a.append(x*o)
            return Quaternion(a,self.w*o)

    def __rmul__(self,o):
        if not isinstance(o,Quaternion) and not isinstance(o,str):
            return self.__mul__(o)
            
    def __imul__(self,o:Quaternion)->Quaternion:
            return self.__mul__(o)
    def __idiv__(self,o:Quaternion)->Quaternion:
            return self.__truediv__(o)
        
    def konjugat(self)->Quaternion:
        a=[]
        for x in self.params:
            a.append(-x)
        return Quaternion(a,self.w)
        
    def norm(self)->float:
        return np.sqrt(self.i**2+self.j**2+self.k**2+self.w**2)
        
    def dot(self,o:Quaternion)->float:
        return self.i*o.i+self.j*o.j+self.k*o.k+self.w*o.w
        
    def inverse(self)->Quaternion:
        q=(self.konjugat()/(self.norm()**2)) 
        return Quaternion(q.params,q.w)
        
    def __str__(self)->str:
        return '['+str(self.i)+'i'+','+str(self.j)+'j'+','+str(self.k)+'k'+','+str(self.w)+']' 
    def __repr__(self)->str:
        return '['+str(self.i)+'i'+','+str(self.j)+'j'+','+str(self.k)+'k'+','+str(self.w)+']'

    def __neg__(self)->Quaternion:
        fi,p=Q2AngleAxis(self)
        return AngleAxis2Q(-fi,-p)

    def __eq__(self,o:Quaternion)->bool:
        return (self.i==o.i and self.j==o.j and self.k==o.k and self.w==o.w and self.params==o.params)

    def to_rotation_matrix(self)->np.array: # Treba prvo normirati vektor pre pozivanja funkcije
        return np.array([[1-2*(self.j**2+self.k**2),2*(self.i*self.j+self.w*self.k),2*(self.i*self.k-self.w*self.j)],
                         [2*(self.i*self.j-self.w*self.k),1-2*(self.i**2+self.k**2),2*(self.j*self.k+self.w*self.i)],
                         [2*(self.i*self.k+self.w*self.j),2*(self.j*self.k-self.w*self.i),1-2*(self.i**2+self.j**2)]])

#dodato zbog predstavljanja SLerpa pomocu stepena (videcemo da li radi)
    def vector_part(self)->Quaternion:
        return (self-self.konjugat())/2
    def sign(self)->Quaternion:
        return self/self.norm()
    def arg(self)->float:
        return np.arccos(w/self.norm())
    def exp(self):
        return (self.vector_part().sign()*np.sin(self.vector_part().norm())+np.cos(self.vector_part().norm()))*math.exp(self.w)
    def ln(self):
        return self.sign()*np.sin(self.vector_part().norm())+math.log(self.norm())
    def __pow__(self,o:int):
        return (self.ln()*o).exp()

In [86]:
#verzija za tester
class Quaternion:
    def __init__(self,params):
        self.i=params[0]
        self.j=params[1]
        self.k=params[2]
        self.w=params[3]
        self.params=params
        
    def __truediv__(self,o:Quaternion)->Quaternion:
        if isinstance(o,Quaternion):
            donji=o.i**2+o.j**2+o.k**2+o.w**2
            return Quaternion([(o.w*self.i-o.i*self.w-o.j*self.k+o.k*self.j)/donji,
                              (o.w*self.j-o.i*self.k-o.j*self.w+o.k*self.i)/donji,
                              (o.w*self.k-o.i*self.j-o.j*self.i+o.k*self.w)/donji],
                              (o.w*self.w-o.i*self.i-o.j*self.j+o.k*self.k)/donji)
        else:
            a=[]
            for x in self.params:
                a.append(x/o)
            return Quaternion(a)
            
    def __add__(self,o:Quaternion)->Quaternion:
        if isinstance(o,Quaternion):
            return Quaternion([self.i+o.i,self.j+o.j,self.k+o.k,self.w+o.w])
        else:
            a=[self.i,self.j,self.k,self.w+o]
            return Quaternion(a)
            
    def __radd__(self,o):
        if not isinstance(o,Quaternion) and not isinstance(o,str):
            return self.__add__(o)

    def __sub__(self,o:Quaternion)->Quaternion:
        if isinstance(o,Quaternion):
            return Quaternion([self.i-o.i,self.j-o.j,self.k-o.k,self.w-o.w])       
        else:
            a=[self.i,self.j,self.k,self.w-o]
            return Quaternion(a)
            
    def __mul__(self,o:Quaternion)->Quaternion:
        if isinstance(o,Quaternion): 
            return Quaternion([self.w*o.i+self.i*o.w+self.j*o.k-self.k*o.j,
                               self.w*o.j+self.j*o.w+self.k*o.i-self.i*o.k,
                               self.w*o.k+self.k*o.w+self.i*o.j-self.j*o.i],
                               self.w*o.w-self.i*o.i-self.j*o.j-self.k*o.k)
        else:
            a=[]
            for x in self.params:
                a.append(x*o)
            return Quaternion(a)

    def __rmul__(self,o):
        if not isinstance(o,Quaternion) and not isinstance(o,str):
            return self.__mul__(o)
            
    def __imul__(self,o:Quaternion)->Quaternion:
            return self.__mul__(o)
    def __idiv__(self,o:Quaternion)->Quaternion:
            return self.__truediv__(o)
        
    def konjugat(self)->Quaternion:
        a=[-self.i,-self.j,-self.k,self.w]
        return Quaternion(a)
        
    def norm(self)->float:
        return np.sqrt(self.i**2+self.j**2+self.k**2+self.w**2)
    def inverse(self)->Quaternion:
        q=(self.konjugat()/(self.norm()**2)) 
        return Quaternion(q.params)

In [7]:
def AngleAxis2Q(fi,p):
    w=np.cos(fi/2)
    p/=np.linalg.norm(p)
    params=np.sin(fi/2)*p
    return Quaternion(params,w)

In [52]:
# def AxisAngle2Q(pphi):  # Ovo je verzija za tester
#     p=[pphi[0],pphi[1],pphi[2]]
#     w=np.cos(pphi[3]/2)
#     p/=np.linalg.norm(p)
#     params=np.sin(pphi[3]/2)*p
#     return np.array([params[0],params[1],params[2],w])

In [72]:
def Q2AngleAxis(q:Quaternion):
    q/=q.norm()
    if q.w<0:
        q=q*(-1)
    fi=2*np.arccos(q.w)
    if abs(q.w) == 1:
        p=np.array[1,0,0]
    else:
        p=q.params
        p/=np.linalg.norm(p)
    return fi,p

In [97]:
def Q2AxisAngle(q): #verzija za tester
    q=Quaternion(q)
    q/=q.norm()
    if q.w<0:
        q=q*(-1)
    fi=2*np.arccos(q.w)
    if abs(q.w) == 1:
        return np.array([1,0,0,0])
    else:
        p=q.params[:-1]
        p/=np.linalg.norm(p)
    pphi=np.array([p[0],p[1],p[2],fi])
 
    pphi = np.where(np.isclose(pphi, 0) , 0 , pphi)  # izbegavanje -0. u rezultatu
    return pphi

In [96]:
print(Q2AxisAngle(np.array([0, 0,0,-3])))

[1 0 0 0]


In [9]:
def Konjugacija(p:Quaternion,q:Quaternion)->Quaternion:
    #Cq(p)=qpq^-1
    return q*p*q.inverse()

In [10]:
def SLerp(q1:Quaternion,q2:Quaternion,tm:int):
    cos0=q1.dot(q2)
    #print(cos0)
    if cos0 < 0:
        q1=q1*(-1)
        cos0=-cos0
    if cos0 > 0.95:
        q1=q1*(-1) #ili lerp interpolaciju
    fi=np.arccos(cos0)
    return [q1*(np.sin(fi*(1-i/tm))/np.sin(fi))
            +q2*(np.sin(fi*(i/tm))/np.sin(fi)) for i in range(tm)]

In [ ]:
q1=Quaternion([3,0,-5],1)
q2=Quaternion([1,1,0],7)

In [ ]:
q7=Quaternion([1,0,2],2)

In [ ]:
(q1/q1.norm()).to_rotation_matrix()

In [ ]:
provera((q1/q1.norm()).to_rotation_matrix())

In [ ]:
q7.norm()

In [ ]:
q8=Quaternion([0,np.sqrt(3)/2,0],1/2)
Konjugacija(Quaternion([0,0,1],0),q8)

In [11]:
q10=Quaternion([1,3,1],-7)

In [12]:
q11=Quaternion([-5,-5,7],1)

In [13]:
q10+q11

[-4i,-2j,8k,-6]

In [14]:
q10*q11

[62i,26j,-38k,6]

In [22]:
q12=Quaternion([1,5,3],1)
6*q12.inverse()

[-0.16666666666666666i,-0.8333333333333334j,-0.5k,0.16666666666666666]

In [21]:
q13=Quaternion([0,np.sqrt(3)/2,0],-(1/2))
ugao,osa=Q2AngleAxis(q13)
ugao=np.degrees(ugao)
ugao
osa

array([-0., -1., -0.])

--------------------------Animacija--------------------------------

In [ ]:
#%pip install manim

In [ ]:
import manim as mm

In [ ]:
mm.interpolate()

In [98]:
import numpy as np
from numpy import linalg
import math
np.set_printoptions(precision=5, suppress=True) 

# pomocne funkcije, ako treba

def AxisAngle2Q(pphi):
    p=[pphi[0],pphi[1],pphi[2]]
    w=np.cos(pphi[3]/2)
    p/=np.linalg.norm(p)
    params=np.sin(pphi[3]/2)*p
    q=np.array([params[0],params[1],params[2],w])
    q = np.where(np.isclose(q, 0) , 0 , q)  # izbegavanje -0. u rezultatu
    return q

In [ ]:
import numpy as np
from numpy import linalg
import math
np.set_printoptions(precision=5, suppress=True) 

#verzija za tester
class Quaternion:
    def __init__(self,params):
        self.i=params[0]
        self.j=params[1]
        self.k=params[2]
        self.w=params[3]
        self.params=params
        
    def __truediv__(self,o):
            a=[]
            for x in self.params:
                a.append(x/o)
            return Quaternion(a)
            
    def __add__(self,o):
            a=[self.i,self.j,self.k,self.w+o]
            return Quaternion(a)
            
    def __sub__(self,o):
            a=[self.i,self.j,self.k,self.w-o]
            return Quaternion(a)
            
    def __mul__(self,o):
            a=[]
            for x in self.params:
                a.append(x*o)
            return Quaternion(a)

            
    def __imul__(self,o):
            return self.__mul__(o)
    def __idiv__(self,o):
            return self.__truediv__(o)
        
    def konjugat(self):
        a=[-self.i,-self.j,-self.k,self.w]
        return Quaternion(a)
        
    def norm(self)->float:
        return np.sqrt(self.i**2+self.j**2+self.k**2+self.w**2)


def Q2AxisAngle(q): #verzija za tester
    q=Quaternion(q)
    q/=q.norm()
    if q.w<0:
        q=q*(-1)
    fi=2*np.arccos(q.w)
    if abs(q.w) == 1:
        return np.array([1,0,0,0])
    else:
        p=q.params[:-1]
        p/=np.linalg.norm(p)
    pphi=np.array([p[0],p[1],p[2],fi])
 
    pphi = np.where(np.isclose(pphi, 0) , 0 , pphi)  # izbegavanje -0. u rezultatu
    return pphi
 
 